In [8]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import requests

#Get the pdf document path
pdf_path = "intro-to-algorithms.pdf"
filename = pdf_path

 #Download the pdf file from the url
if not os.path.exists(pdf_path):
    print(f"[INFO] File does not exist, downloading...")
    
    url = "https://www.cs.mcgill.ca/~akroit/math/compsci/Cormen%20Introduction%20to%20Algorithms.pdf"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        #Open the file and save it
        with open(filename, "wb") as file:
            file.write(response.content)
            
        print(f"[INFO] The file has been downloaded and saved successfully as {filename}")
        
    else:
        print(f"[ERROR] Failed to download the file. Status Code: {response.status_code}")
else:
    print(f"[INFO] The file already exists: {filename}")




[INFO] The file already exists: intro-to-algorithms.pdf


We now need to open the PDF

In [10]:
%pip install PyMuPDF 
%pip install tqdm
%pip install re

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
Note: you may need to restart the kernel to use updated packages.


In [11]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    """ Performs minor formatting on text """
    text = text.replace("\n", " ")
    return text.strip()

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """ Extracts text from a PDF file and reads its contents """
    doc = fitz.open(pdf_path)
    pages_and_text = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_text.append({
            "page_number": page_number - 20,
            "page_char_count": len(text),
            "page_word_count": len(text.split(" ")),
            "page_sentence_count": len(text.split('.')),
            "page_token_count": len(text) / 4, #1 token = ~ 4 characters
            "text": text
        })
    return pages_and_text

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]
    
    
    

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
1313it [00:02, 576.05it/s]


[{'page_number': -20,
  'page_char_count': 199,
  'page_word_count': 105,
  'page_sentence_count': 4,
  'page_token_count': 49.75,
  'text': 'A L G O R I T H M S I N T R O D U C T I O N  T O T H I R D  E D I T I O N T H O M A S  H.  C H A R L E S  E.       R O N A L D  L.  C L I F F O R D S T E I N R I V E S T L E I S E R S O N C O R M E N'},
 {'page_number': -19,
  'page_char_count': 40,
  'page_word_count': 5,
  'page_sentence_count': 1,
  'page_token_count': 10.0,
  'text': 'Introduction to Algorithms Third Edition'}]

In [12]:
import random

random.sample(pages_and_texts, k=3)

[{'page_number': 1227,
  'page_char_count': 1360,
  'page_word_count': 305,
  'page_sentence_count': 24,
  'page_token_count': 340.0,
  'text': 'Problems for Appendix D 1227 is det.V.x0; x1; : : : ; xn\x001// D Y 0\x14j <k\x14n\x001 .xk \x00 xj/ : (Hint: Multiply column i by \x00x0 and add it to column i C 1 for i D n \x00 1; n \x00 2; : : : ; 1, and then use induction.) D-2 Permutations deﬁned by matrix-vector multiplication over GF.2/ One class of permutations of the integers in the set Sn D f0; 1; 2; : : : ; 2n \x00 1g is deﬁned by matrix multiplication over GF.2/. For each integer x in Sn, we view its binary representation as an n-bit vector \x00x0 x1 x2 ::: xn\x001 \x01 ; where x D Pn\x001 iD0 xi2i. If A is an n \x02 n matrix in which each entry is either 0 or 1, then we can deﬁne a permutation mapping each value x in Sn to the number whose binary representation is the matrix-vector product Ax. Here, we perform all arithmetic over GF.2/: all values are either 0 or 1, and with one 

In [13]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd

df = pd.DataFrame(pages_and_text)
df.head()


NameError: name 'pages_and_text' is not defined

In [ ]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count
count,1313.00,1313.00,1313.00,1313.0,1313.00
mean,636.00,1935.34,365.27,1.0,483.83
std,379.17,577.43,101.17,0.0,144.36
min,-20.00,0.00,1.00,1.0,0.00
25%,308.00,1603.00,310.00,1.0,400.75
50%,636.00,1974.00,370.00,1.0,493.50
75%,964.00,2334.00,434.00,1.0,583.50
max,1292.00,3275.00,639.00,1.0,818.75


Why would we care about token count? 

Token count is important to think about because: 
1. Embedding models don't deal with infinite tokens.
2. LLMs don't deal with infinite tokens.

For example, an embedding model may have been trained to embed sequences of 384 tokens into numerical space (sentence transformers `all-mpnet-base-v2`, see: https://www.sbert.net/docs/pretrained_models.html).

As for LLMs, they can't accept infinite tokens in their context window. 

### Further text processing (Splitting pages into sentences)

Two ways to do this: 
1. We've done this by splitting on `"."`. 
2. We can do this with an NLP library such as [spaCy](https://spacy.io/usage) and [nltk](https://nltk.org/).

Note to Self: spacy was downloaded and not added into the Notebook. Should be installed based on the instructions provided in spacy.io.

In [ ]:
from spacy.lang.en import English

nlp = English()

#Add a sentencizer pipeline, see https://spacy.io/api/sentencizer
nlp.add_pipe("sentencizer")

#Create document instance as an example
doc = nlp("This is a sentence. This is another sentence. I like Elephants.")
assert len(list(doc.sents)) == 3

# Print out our sentences split
list(doc.sents)

[This is a sentence., This is another sentence., I like Elephants.]

In [20]:
for item in tqdm(pages_and_texts): 
    item["sentences"] = list(nlp(item["text"]).sents) # Group text into sentences and add to pages and texts
    
    # Make sure all sentences are strings (the default type is a spaCy datatype)
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    
    # Count the sentences 
    item["page_sentence_count_spacy"] = len(item["sentences"])

100%|██████████| 1313/1313 [00:01<00:00, 1143.01it/s]


In [22]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,page_sentence_count_spacy
count,1313.00,1313.00,1313.00,1313.00,1313.00,1313.00
mean,636.00,1935.34,365.27,30.08,483.83,14.71
std,379.17,577.43,101.17,13.90,144.36,8.01
min,-20.00,0.00,1.00,1.00,0.00,0.00
25%,308.00,1603.00,310.00,22.00,400.75,10.00
50%,636.00,1974.00,370.00,28.00,493.50,14.00
75%,964.00,2334.00,434.00,35.00,583.50,18.00
max,1292.00,3275.00,639.00,111.00,818.75,68.00


### Chunking our sentences together

The concept of splitting larger pieces of text into smaller ones is often referred to as text splitting or chunking. 

There is no 100% correct way of doing this. 

To keep it simple, we will split into groups of 10 sentences (however, can also try with 5, 7 or 8, whatever works).

There are frameworks such as [LangChain](https://python.langchain.com/docs/concepts/text_splitters/) that can help with this, but we will stick for Python for this tutorial.

Why we do this:
1. So our texts are easier to filter (smaller groups of text can be easier to inspect than larger passages of text)
2. So our text chunks can fit into our embedding model context window
3. So our contexts passed to an LLM can be more specific and focused.  

In [24]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 10

# Create a function to split lists of texts recursively into chunk size
# e.g. [20] -> [10, 10], [25] -> [10, 10, 5]
def split_list(input_list: list, 
               slice_size: int=num_sentence_chunk_size) -> list[list[str]]:
    return [input_list[i:i+slice_size] for i in range(0, len(input_list), slice_size)]
test_list = list(range(25))
split_list(test_list) 


[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [25]:
# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"]) 
    

100%|██████████| 1313/1313 [00:00<00:00, 548736.66it/s]


In [26]:
random.sample(pages_and_texts, k=1)

[{'page_number': 667,
  'page_char_count': 1771,
  'page_word_count': 329,
  'page_sentence_count': 46,
  'page_token_count': 442.75,
  'text': '24.4 Difference constraints and shortest paths 667 0 0 0 0 0 0 –1 1 5 4 –1 –3 –3 0 –5 –3 0 –1 –4 v3 v2 v1 v5 v0 v4 Figure 24.8 The constraint graph corresponding to the system (24.3)–(24.10) of difference con- straints. The value of ı.\x170; \x17i/ appears in each vertex \x17i. One feasible solution to the system is x D .\x005; \x003; 0; \x001; \x004/. The constraint graph contains the additional vertex \x170, as we shall see shortly, to guarantee that the graph has some vertex which can reach all other vertices. Thus, the vertex set V consists of a vertex \x17i for each unknown xi, plus an additional vertex \x170. The edge set E contains an edge for each difference constraint, plus an edge .\x170; \x17i/ for each unknown xi. If xj \x00 xi \x14 bk is a difference constraint, then the weight of edge .\x17i; \x17j/ is w.\x17i; \x17j/ D bk. The w

In [28]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,page_sentence_count_spacy,num_chunks
count,1313.00,1313.00,1313.00,1313.00,1313.00,1313.00,1313.00
mean,636.00,1935.34,365.27,30.08,483.83,14.71,1.92
std,379.17,577.43,101.17,13.90,144.36,8.01,0.84
min,-20.00,0.00,1.00,1.00,0.00,0.00,0.00
25%,308.00,1603.00,310.00,22.00,400.75,10.00,1.00
50%,636.00,1974.00,370.00,28.00,493.50,14.00,2.00
75%,964.00,2334.00,434.00,35.00,583.50,18.00,2.00
max,1292.00,3275.00,639.00,111.00,818.75,68.00,7.00


### Splitting each chunk into its own item

We'd like to embed each chunk of sentences into its own numerical representation. 

That'll give us a good level of granularity.

Meaning, we can dive specifically into the text sample that was used in our model.

In [32]:
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        
        # Join the sentences into a paragraph like structure, aka join the list of sentences into one paragraph
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
        
        chunk_dict["sentence_chunk"] = joined_sentence_chunk
        
        # Get some stats on our chunks
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~ 4 chars
        
        pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)
        
        

100%|██████████| 1313/1313 [00:00<00:00, 32985.07it/s]


2517

In [33]:
random.sample(pages_and_chunks, k = 1)

[{'page_number': 876,
  'sentence_chunk': '876 Chapter 29 Linear Programming Lemma 29.3 Let I be a set of indices. For each j 2 I, let ˛j and ˇj be real numbers, and let xj be a real-valued variable. Let be any real number. Suppose that for any settings of the xj, we have X j2I ˛jxj D C X j2I ˇjxj : (29.78) Then ˛j D ˇj for each j 2 I, and D 0. Proof Since equation (29.78) holds for any values of the xj, we can use particular values to draw conclusions about ˛, ˇ, and . If we let xj D 0 for each j 2 I, we conclude that D 0. Now pick an arbitrary index j 2 I, and set xj D 1 and xk D 0 for all k ¤ j . Then we must have ˛j D ˇj. Since we picked j as any index in I, we conclude that ˛j D ˇj for each j 2 I. A particular linear program has many different slack forms; recall that each slack form has the same set of feasible and optimal solutions as the original linear pro- gram. We now show that the slack form of a linear program is uniquely determined by the set of basic variables.',
  'chun

In [34]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,2517.00,2517.00,2517.00,2517.00
mean,669.58,1008.50,189.95,252.12
std,390.17,515.08,105.23,128.77
min,-20.00,9.00,2.00,2.25
25%,340.00,585.00,100.00,146.25
50%,668.00,1046.00,191.00,261.50
75%,1018.00,1381.00,263.00,345.25
max,1292.00,2607.00,548.00,651.75


### Filter chunks of text for short chunks

These chunks may not contain much useful information.

In [40]:
# Show random chunks with under 30 tokens in length
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk Token Count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk Token Count: 14.0 | Text: The expected value of the random variable X representing
Chunk Token Count: 8.25 | Text: The total time is thus O.n lg n/.
Chunk Token Count: 11.75 | Text: 505 insertion into binary search trees, 294–295
Chunk Token Count: 6.5 | Text: V Advanced Data Structures
Chunk Token Count: 4.5 | Text: The relation <x is


In [41]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': -20,
  'sentence_chunk': 'A L G O R I T H M S I N T R O D U C T I O N T O T H I R D E D I T I O N T H O M A S H. C H A R L E S E.    R O N A L D L. C L I F F O R D S T E I N R I V E S T L E I S E R S O N C O R M E N',
  'chunk_char_count': 189,
  'chunk_word_count': 95,
  'chunk_token_count': 47.25},
 {'page_number': -17,
  'sentence_chunk': 'Thomas H. Cormen Charles E. Leiserson Ronald L. Rivest Clifford Stein Introduction to Algorithms Third Edition The MIT Press Cambridge, Massachusetts London, England',
  'chunk_char_count': 165,
  'chunk_word_count': 23,
  'chunk_token_count': 41.25}]

In [45]:
random.sample(pages_and_chunks_over_min_token_len, k=1)

[{'page_number': 236,
  'sentence_chunk': 'In a circular list, the pre\x17 pointer of the head of the list points to the tail, and the next pointer of the tail of the list points to the head. We can think of a circular list as a ring of',
  'chunk_char_count': 190,
  'chunk_word_count': 42,
  'chunk_token_count': 47.5}]

# Embedding Our Text Chunks

[Embeddings](https://vickiboykis.com/what_are_embeddings/) are a broad but powerful concept.

While humans understand text, machines understand numbers. 

What we'd like to do: 
- Turn our text chunks into numbers, specifically, embeddings. 

A useful numerical representation.

The best part about embeddings is that they are a *learned* representation.

```
{"the": 0,
"a": 1, 
...
...}
```

In [50]:
%pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cpu")

# Create a list of sentences
sentences = ["Hello, my name is John and I like football.",
             "AI is so cool nowadays.",
             "Embeddings are a revolutionary technology",
             "I went to the farm the other day."]

embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences, embeddings))

# See the embeddings
for sentence, embedding in embeddings_dict.items():
    print(f"Sentence: {sentence}")
    print(f"Embedding: {embedding}")
    print("")

Note: you may need to restart the kernel to use updated packages.
Sentence: Hello, my name is John and I like football.
Embedding: [ 1.29194220e-03  7.20893592e-02 -1.17977790e-03 -2.62890253e-02
  3.96553315e-02 -4.10587005e-02 -5.22783026e-02  7.64403446e-03
 -1.27780419e-02 -1.90602019e-04  3.95361073e-02 -2.21871939e-02
  1.24161486e-02  1.51262730e-02  3.52668054e-02 -4.13680039e-02
  4.56340332e-03 -6.52403831e-02  6.32882938e-02 -7.24698836e-03
  4.18687873e-02  5.12758212e-04 -1.46308355e-02 -1.39766848e-02
 -5.79854287e-02  2.54586097e-02 -1.36849908e-02  4.45831427e-03
 -1.05274487e-02  7.11089149e-02 -9.09978990e-03 -3.25113684e-02
 -3.53915337e-03 -1.13454703e-02  1.58650278e-06  1.18198972e-02
 -1.19213499e-02 -2.13643741e-02  7.80989155e-02  2.57006002e-04
  1.19429715e-02 -3.15609947e-03 -1.37991933e-02 -3.18820425e-03
 -8.89387634e-03  6.08826894e-03  5.48814870e-02  6.89790631e-03
  1.10323401e-02  3.49150077e-02 -6.11934531e-03 -3.25804055e-02
 -9.72216111e-03 -1.9384

In [55]:
%%time
import torch

device = "mps" if torch.backends.mps.is_available() else "cpu"
embedding_model.to(device)

#Embed each chunk one by one
for item in tqdm(pages_and_chunks_over_min_token_len): 
    item["embedding"] = embedding_model.encode(item["sentence_chunk"]) 

100%|██████████| 2441/2441 [02:15<00:00, 17.96it/s]

CPU times: user 55.7 s, sys: 12.9 s, total: 1min 8s
Wall time: 2min 16s


In [56]:
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]
text_chunks[419]

'The elements in the queue reside in locations Q:head; Q:head C 1; : : : ; Q:tail \x00 1, where we “wrap around” in the sense that location 1 immediately follows location n in a circular order. When Q:head D Q:tail, the queue is empty. Initially, we have Q:head D Q:tail D 1. If we attempt to dequeue an element from an empty queue, the queue underﬂows.'

In [57]:
len(text_chunks)

2441

In [59]:
%%time

#Embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks, 
                                                batch_size = 32, # Batch Size -> How many chunks to look at, at one time. Can experiment to see which batch size leads to best results
                                                convert_to_tensor=True)
text_chunk_embeddings

CPU times: user 3.3 s, sys: 2.38 s, total: 5.68 s
Wall time: 58.6 s


tensor([[-0.0655, -0.0349, -0.0133,  ...,  0.0538, -0.0036,  0.0226],
        [-0.0205, -0.0198, -0.0266,  ..., -0.0105,  0.0293,  0.0002],
        [-0.0261,  0.0296, -0.0303,  ...,  0.0027, -0.0106, -0.0087],
        ...,
        [-0.0291, -0.0882, -0.0420,  ..., -0.0110, -0.0173, -0.0282],
        [-0.0380, -0.0687, -0.0491,  ..., -0.0103, -0.0122, -0.0500],
        [-0.0146,  0.0466, -0.0223,  ...,  0.0301,  0.0003, -0.0313]],
       device='mps:0')